In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

In [2]:
from google.colab import files
uploaded = files.upload()

Saving nhes_19_pfi_cleaned.csv to nhes_19_pfi_cleaned (3).csv


In [3]:
# Import our input dataset
import io
learning_data_df = pd.read_csv(io.BytesIO(uploaded['nhes_19_pfi_cleaned.csv']))
learning_data_df.head()

,Unnamed: 0,ALLGRADEX_2,ALLGRADEX_3,ALLGRADEX_4,ALLGRADEX_5,ALLGRADEX_6,ALLGRADEX_7,ALLGRADEX_8,ALLGRADEX_9,ALLGRADEX_10,ALLGRADEX_11,ALLGRADEX_12,ALLGRADEX_13,ALLGRADEX_14,ALLGRADEX_15,SEGRADES_0,SEGRADES_1,CENREG_1,CENREG_2,CENREG_3,CENREG_4,EDCPUB_1,EDCPUB_2,EDCCAT_1,EDCCAT_2,EDCREL_1,EDCREL_2,EDCPRI_1,EDCPRI_2,EDCINTK12_1,EDCINTK12_2,EDCHSFL_1,EDCHSFL_2,DISTASSI_1,DISTASSI_2,SCHRTSCHL_1,SCHRTSCHL_2,SCHLMAGNET_1,SCHLMAGNET_2,SEENJOY_1,...,NUMSIBSX_1,NUMSIBSX_2,NUMSIBSX_3,NUMSIBSX_4,NUMSIBSX_5,NUMSIBSX_6,NUMSIBSX_7,TTLHHINC_1,TTLHHINC_2,TTLHHINC_3,TTLHHINC_4,TTLHHINC_5,TTLHHINC_6,TTLHHINC_7,TTLHHINC_8,TTLHHINC_9,TTLHHINC_10,TTLHHINC_11,TTLHHINC_12,OWNRNTHB_1,OWNRNTHB_2,OWNRNTHB_3,HVINTSPHO_1,HVINTSPHO_2,HVINTCOM_1,HVINTCOM_2,INTACC_1,INTACC_2,INTACC_3,INTACC_4,CHLDNT_1,CHLDNT_2,CHLDNT_3,CHLDNT_4,LRNCOMP_1,LRNCOMP_2,LRNTAB_1,LRNTAB_2,LRNCELL_1,LRNCELL_2
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0
1,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0,1
2,7,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,1,...,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0
3,9,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,1,0,0,1
4,11,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,1,...,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0


In [4]:
# remove allgrade columns and preserve in a separate df
allgrade_df = learning_data_df.filter(['ALLGRADEX_2',
 'ALLGRADEX_3',
 'ALLGRADEX_4',
 'ALLGRADEX_5',
 'ALLGRADEX_6',
 'ALLGRADEX_7',
 'ALLGRADEX_8',
 'ALLGRADEX_9',
 'ALLGRADEX_10',
 'ALLGRADEX_11',
 'ALLGRADEX_12',
 'ALLGRADEX_13',
 'ALLGRADEX_14',
 'ALLGRADEX_15'], axis=1)
allgrade_df.head()

,ALLGRADEX_2,ALLGRADEX_3,ALLGRADEX_4,ALLGRADEX_5,ALLGRADEX_6,ALLGRADEX_7,ALLGRADEX_8,ALLGRADEX_9,ALLGRADEX_10,ALLGRADEX_11,ALLGRADEX_12,ALLGRADEX_13,ALLGRADEX_14,ALLGRADEX_15
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [5]:
# remove allgrade columns from learning_data_df
learning_data_df = learning_data_df.drop([ 'ALLGRADEX_2',
 'ALLGRADEX_3',
 'ALLGRADEX_4',
 'ALLGRADEX_5',
 'ALLGRADEX_6',
 'ALLGRADEX_7',
 'ALLGRADEX_8',
 'ALLGRADEX_9',
 'ALLGRADEX_10',
 'ALLGRADEX_11',
 'ALLGRADEX_12',
 'ALLGRADEX_13',
 'ALLGRADEX_14',
 'ALLGRADEX_15'], axis=1)

In [6]:
# Split our preprocessed data into our features and target arrays
y = learning_data_df['SEGRADES_1']
X = learning_data_df.drop(['SEGRADES_0', 'Unnamed: 0','SEGRADES_1'], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [7]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
X_train

,CENREG_1,CENREG_2,CENREG_3,CENREG_4,EDCPUB_1,EDCPUB_2,EDCCAT_1,EDCCAT_2,EDCREL_1,EDCREL_2,EDCPRI_1,EDCPRI_2,EDCINTK12_1,EDCINTK12_2,EDCHSFL_1,EDCHSFL_2,DISTASSI_1,DISTASSI_2,SCHRTSCHL_1,SCHRTSCHL_2,SCHLMAGNET_1,SCHLMAGNET_2,SEENJOY_1,SEENJOY_2,SEENJOY_3,SEENJOY_4,SEABSNT_1,SEABSNT_2,SEABSNT_3,SEABSNT_4,FCSCHOOL_1,FCSCHOOL_2,FCSCHOOL_3,FCSCHOOL_4,FCTEACHR_1,FCTEACHR_2,FCTEACHR_3,FCTEACHR_4,FCSTDS_1,FCSTDS_2,...,NUMSIBSX_1,NUMSIBSX_2,NUMSIBSX_3,NUMSIBSX_4,NUMSIBSX_5,NUMSIBSX_6,NUMSIBSX_7,TTLHHINC_1,TTLHHINC_2,TTLHHINC_3,TTLHHINC_4,TTLHHINC_5,TTLHHINC_6,TTLHHINC_7,TTLHHINC_8,TTLHHINC_9,TTLHHINC_10,TTLHHINC_11,TTLHHINC_12,OWNRNTHB_1,OWNRNTHB_2,OWNRNTHB_3,HVINTSPHO_1,HVINTSPHO_2,HVINTCOM_1,HVINTCOM_2,INTACC_1,INTACC_2,INTACC_3,INTACC_4,CHLDNT_1,CHLDNT_2,CHLDNT_3,CHLDNT_4,LRNCOMP_1,LRNCOMP_2,LRNTAB_1,LRNTAB_2,LRNCELL_1,LRNCELL_2
6838,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0
5292,0,0,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,1,0,1,0,0,1
5815,0,0,0,1,1,0,0,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,1,1,0,1,0
2569,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,1,1,0,0,1
4268,0,0,0,1,1,0,0,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6249,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,...,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0
4136,1,0,0,0,1,0,0,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0
6039,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,1,1,0,1,0
470,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0


In [9]:
# Define the model - deep neural net
number_input_features = 186
hidden_nodes_layer1 = 200
hidden_nodes_layer2 = 100

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               37400     
                                                                 
 dense_1 (Dense)             (None, 100)               20100     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 57,601
Trainable params: 57,601
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [11]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=50)

Epoch 1/50
196/196 [==============================] - 2s 4ms/step - loss: 0.6650 - accuracy: 0.6458
Epoch 2/50
196/196 [==============================] - 1s 4ms/step - loss: 0.5931 - accuracy: 0.6826
Epoch 3/50
196/196 [==============================] - 1s 4ms/step - loss: 0.5754 - accuracy: 0.7112
Epoch 4/50
196/196 [==============================] - 1s 4ms/step - loss: 0.5651 - accuracy: 0.7152
Epoch 5/50
196/196 [==============================] - 1s 4ms/step - loss: 0.5601 - accuracy: 0.7189
Epoch 6/50
196/196 [==============================] - 1s 4ms/step - loss: 0.6789 - accuracy: 0.6732
Epoch 7/50
196/196 [==============================] - 1s 4ms/step - loss: 0.5725 - accuracy: 0.7093
Epoch 8/50
196/196 [==============================] - 1s 4ms/step - loss: 0.5549 - accuracy: 0.7262
Epoch 9/50
196/196 [==============================] - 1s 4ms/step - loss: 0.5303 - accuracy: 0.7458
Epoch 10/50
196/196 [==============================] - 1s 4ms/step - loss: 0.4944 - accuracy: 0.7585

In [12]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

66/66 - 0s - loss: 3.1236 - accuracy: 0.6257 - 250ms/epoch - 4ms/step
Loss: 3.123619318008423, Accuracy: 0.6256582140922546
